# Notebook used to perform fixed pwm analysis

## Imports

In [ ]:
from thermalframe import ThermalFrame
from thermalutil import generateThermalVideo
from InfluxDBInterface.libdb import download_tmp_DB

import pandas as pd
import matplotlib.pyplot as plt

## Configuration

In [ ]:
# Start and duration of the 'experiments'
dts = {
    'square24': '2025-09-25T15:16:20Z',
    'gauss24': '2025-09-25T15:37:20Z',
    'slope24': '2025-09-25T15:54:00Z',
    'square13': '2025-09-26T08:25:00Z',
    'gauss13': '2025-09-26T08:46:00Z',
    'slope13': '2025-09-26T09:45:00Z'
}

## Downloading required data

In [ ]:
# Convert ts to pd.Timestamps
for k in dts:
    dts[k] = pd.Timestamp(dts[k])

# Find the range of timestamps to download
all_timestamps = list(dts.values())
t_start = min(all_timestamps) - pd.Timedelta('10min')
t_end = max(all_timestamps) + pd.Timedelta('30min')

# Download data from InfluxDB
filters = {'hive_num': 0, 'inhive_loc': 'none'}
tmp_df = download_tmp_DB('meb_dev', t_start, t_end, 20, filters)
print(f"Downloaded tmp data from Influx DB from {t_start} to {t_end} with {len(tmp_df)} entries.")
print(tmp_df.head())
print(tmp_df.shape)

# Make a plot with dots where we have data
fig, ax = plt.subplots(1, 1, figsize=(10, 2))
ax.scatter(tmp_df.index, [1] * len(tmp_df), s=1)
ax.set_xlabel('Time')
ax.set_title('Temperature Data Availability')
ax.set_yticks([0, 1])
plt.show()

## Plotting

In [ ]:
square24_df = tmp_df.loc[dts['square24']+pd.Timedelta('10min')]

square_24 = square24_df.to_numpy()
tf = ThermalFrame(square_24, hive_id=0)
fig, ax = plt.subplots(figsize=(9, 5))
ax = tf.plot_thermal_field(ax=ax,
                           show_cb=True,
                           show_max_temp=True, 
                           show_sensors=True)
plt.show()

## Video generation

In [ ]:
# Now we make videos for all 6 exps
duration = ['22min', '16min', '30min', '21min', '21min', '25min']
for i, (key, value) in enumerate(dts.items()):
    print(f"----Generating video for {key}----")
    df_therm_data = tmp_df.loc[value:value+pd.Timedelta(duration[i]),:]
    generateThermalVideo(df_therm_data, key, "outputVideos/fixedPWM", 
                            fps=10, 
                            show_cb=True,
                            show_sensors=True,
                            show_max_temp=True,
                            verbose=False)
